In [3]:
from dotenv import load_dotenv

load_dotenv("openai.env")

True

# 如何给agent正确的增加记忆

- 将memory插入到提示词模板中
<hr>

In [4]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.chains import LLMMathChain
from langchain.prompts import MessagesPlaceholder

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)


自定义tools

In [5]:
import os

os.environ["SERPAPI_API_KEY"] = "32109af1224c43d11768866970c41e4e681e03bf220c7bd2b9a16af817431719"
#构建一个搜索工具
search = SerpAPIWrapper()
#创建一个数学计算工具
llm_math_chain = LLMMathChain(
    llm=llm,
    verbose=True
)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]
print(tools)

[Tool(name='Search', description='useful for when you need to answer questions about current events or the current state of the world', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='32109af1224c43d11768866970c41e4e681e03bf220c7bd2b9a16af817431719', aiosession=None)>), Tool(name='Calculator', description='useful for when you need to answer questions about math', func=<bound method Chain.run of LLMMathChain(verbose=True, llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n

/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


增加memory组件

In [6]:
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

定义agent

In [7]:
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    handle_parsing_errors=True,#处理解析错误
    memory=memory #记忆组件
)

/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [8]:
print(agent_chain.agent.prompt.messages)
print(agent_chain.agent.prompt.messages[0])
print(agent_chain.agent.prompt.messages[1])
print(agent_chain.agent.prompt.messages[2])

[SystemMessage(content='You are a helpful AI assistant.'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]
content='You are a helpful AI assistant.'
prompt=PromptTemplate(input_variables=['input'], template='{input}')
variable_name='agent_scratchpad'


In [9]:
agent_chain.invoke("你好,我是大头")



> Entering new AgentExecutor chain...
你好大头！有什么我可以帮助你的吗？

> Finished chain.


{'input': '你好,我是大头',
 'chat_history': [HumanMessage(content='你好,我是大头'),
  AIMessage(content='你好大头！有什么我可以帮助你的吗？')],
 'output': '你好大头！有什么我可以帮助你的吗？'}

In [10]:
# 可以看到memory没有生效, 
agent_chain.invoke("我叫什么名字?")



> Entering new AgentExecutor chain...
作为一个AI，我没有访问您个人信息的权限，所以我不知道您的名字。如果您愿意告诉我，我会知道如何称呼您。

> Finished chain.


{'input': '我叫什么名字?',
 'chat_history': [HumanMessage(content='你好,我是大头'),
  AIMessage(content='你好大头！有什么我可以帮助你的吗？'),
  HumanMessage(content='我叫什么名字?'),
  AIMessage(content='作为一个AI，我没有访问您个人信息的权限，所以我不知道您的名字。如果您愿意告诉我，我会知道如何称呼您。')],
 'output': '作为一个AI，我没有访问您个人信息的权限，所以我不知道您的名字。如果您愿意告诉我，我会知道如何称呼您。'}

需要使用agent_kwargs传递参数，将chat_history传入

In [11]:
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    handle_parsing_errors=True,#处理解析错误
    agent_kwargs={
        "extra_prompt_messages":[
             MessagesPlaceholder(variable_name="chat_history")
            # 下面这个是原来就存在的占位符,我们也需要带上
            ,MessagesPlaceholder(variable_name="agent_scratchpad")
        ],
    },
    memory=memory #记忆组件
)

查看重新渲染后的提示词模版

In [12]:
print(agent_chain.agent.prompt.messages)
print(agent_chain.agent.prompt.messages[0])
print(agent_chain.agent.prompt.messages[1])
print(agent_chain.agent.prompt.messages[2])

[SystemMessage(content='You are a helpful AI assistant.'), MessagesPlaceholder(variable_name='chat_history'), MessagesPlaceholder(variable_name='agent_scratchpad'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]
content='You are a helpful AI assistant.'
variable_name='chat_history'
variable_name='agent_scratchpad'


In [14]:
agent_chain.invoke("好厉害，刚才我们都聊了什么？")



> Entering new AgentExecutor chain...
刚才我们的对话主要是您向我打招呼，告诉我您的昵称是大头，然后询问我是否知道您的名字。我回答说没有访问您个人信息的权限，所以不知道您的名字。如果您有其他问题或需要帮助，请随时告诉我！

> Finished chain.


{'input': '好厉害，刚才我们都聊了什么？',
 'chat_history': [HumanMessage(content='你好,我是大头'),
  AIMessage(content='你好大头！有什么我可以帮助你的吗？'),
  HumanMessage(content='我叫什么名字?'),
  AIMessage(content='作为一个AI，我没有访问您个人信息的权限，所以我不知道您的名字。如果您愿意告诉我，我会知道如何称呼您。'),
  HumanMessage(content='好厉害，刚才我们都聊了什么？'),
  AIMessage(content='刚才我们的对话主要是您向我打招呼，告诉我您的昵称是大头，然后询问我是否知道您的名字。我回答说没有访问您个人信息的权限，所以不知道您的名字。如果您有其他问题或需要帮助，请随时告诉我！')],
 'output': '刚才我们的对话主要是您向我打招呼，告诉我您的昵称是大头，然后询问我是否知道您的名字。我回答说没有访问您个人信息的权限，所以不知道您的名字。如果您有其他问题或需要帮助，请随时告诉我！'}